In [1]:
import torch
import torch.optim as optim
import os
import yaml
# import wandb

from jinja2 import Environment, FileSystemLoader

from training.create_dataset import *
from training.create_network import *
from training.utils import create_task_flags, TaskMetric, eval
from utils import torch_save, get_data_loaders, initialize_wandb

# Login to wandb
# wandb.login()

In [2]:
# Options for training
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('config/mtl.yaml.j2')
rendered_yaml = template.render()
config = yaml.safe_load(rendered_yaml)

# Create logging folder to store training weights and losses
os.makedirs("logs", exist_ok=True)

model_classes = {
  "split": MTLDeepLabv3,
  "mtan": MTANDeepLabv3,
  # "dinov2": MTLDinoVisionTransformer,
}

In [3]:
# initialize_wandb(
#   project=config["wandb"]["project"], 
#   group=f"{config['training_params']['network']}", 
#   job_type="task_specific", 
#   mode=config["wandb"]["mode"], 
#   config={
#     "task": config['training_params']['task'],
#     "network": config['training_params']['network'],
#     "dataset": config['training_params']['dataset'],
#     "epochs": config['training_params']['total_epochs'],
#     "lr": config['training_params']['lr'],
#     "batch_size": config['training_params']['batch_size'],
#     "seed": config['training_params']['seed'],
#   }
# )

In [3]:
torch.manual_seed(config["training_params"]["seed"])
np.random.seed(config["training_params"]["seed"])
random.seed(config["training_params"]["seed"])

# device = torch.device(f"cuda:{config["training_params"]['gpu']}" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
train_loader, val_loader, test_loader = get_data_loaders(config)

In [5]:
from models.dinov2.mtl.multitasker import MTLDinoV2

# Initialize model
train_tasks = create_task_flags(config["training_params"]["task"], config["training_params"]["dataset"])
model = MTLDinoV2(
  arch_name="vit_small",
  head_tasks=train_tasks,
)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)


/Users/juangarcia/Dropbox/Course/Year 2/Semester 1/Semester Project/repositories/simomm/models/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/Users/juangarcia/Dropbox/Course/Year 2/Semester 1/Semester Project/repositories/simomm/models/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/Users/juangarcia/Dropbox/Course/Year 2/Semester 1/Semester Project/repositories/simomm/models/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
/Users/juangarcia/Dropbox/Course/Year 2/Semester 1/Semester Project/repositories/simomm/models/dinov2/losses/cross_entropy_loss.py:220: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cr

In [6]:
print(f"Model: {config['training_params']['network'].title()} | Number of Trainable Parameters: {num_params/1e6:.2f}M")
print(f"Training Task: {config['training_params']['dataset'].title()} - {config['training_params']['task'].title()} in Single Task Learning Mode with {config['training_params']['network'].upper()}")

Model: Split | Number of Trainable Parameters: 22.87M
Training Task: Nyuv2 - All in Single Task Learning Mode with SPLIT


In [7]:
model.freeze_shared_layers()
optimizer = optim.AdamW(model.parameters(), lr=config["training_params"]["lr"], weight_decay=1e-4)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=config["training_params"]["lr"], steps_per_epoch=len(train_loader), epochs=config["training_params"]["total_epochs"],  pct_start=0.1)

In [10]:
train_batch = len(train_loader)
test_batch = len(test_loader)
train_metric = TaskMetric(train_tasks, train_tasks, config["training_params"]["batch_size"], config["training_params"]["total_epochs"], config["training_params"]["dataset"])
test_metric = TaskMetric(train_tasks, train_tasks, config["training_params"]["batch_size"], config["training_params"]["total_epochs"], config["training_params"]["dataset"])

In [11]:
#  Training loop
for epoch in range(config["training_params"]["total_epochs"]):

    # training
    model.train()
    train_dataset = iter(train_loader)
    for k in range(train_batch):
        train_data, train_target = next(train_dataset)
        train_data = train_data.to(device)
        train_target = {task_id: train_target[task_id].to(device) for task_id in train_tasks.keys()}
        
        train_res = model(train_data, None, img_gt=train_target, return_loss=True)
        
        optimizer.zero_grad()
        train_res["total_loss"].backward()
        optimizer.step()

        train_metric.update_metric(train_res, train_target)
        
        if k == 5:
          break

    train_str = train_metric.compute_metric()
    # wandb.log({
    #     **{f"train/loss/{task_id}": task_data["total_loss"] for task_id, task_data in train_res.items()},
    #     **{f"train/metric/{task_id}": train_metric.get_metric(task_id) for task_id in train_tasks}
    # }, step=epoch)
    train_metric.reset()

    # evaluating
    test_str = eval(epoch, model, test_loader, test_metric)

    scheduler.step()

    print(f"Epoch {epoch:04d} | TRAIN:{train_str} || TEST:{test_str} | Best: {config['training_params']['task'].title()} {test_metric.get_best_performance(config['training_params']['task']):.4f}")

    # task_dict = {"train_loss": train_metric.metric, "test_loss": test_metric.metric}
    # np.save("logging/stl_{}_{}_{}_{}.npy".format(config["training_params"]["network"], config["training_params"]["dataset"], config["training_params"]["task"], config["training_params"]["seed"]), task_dict)
    torch_save(model, "checkpoints/dinov2/linear_probing/seg_model.pt")

 Seg 2.0845 0.1315 Depth 0.0793 0.8321


NameError: name 'afs' is not defined

In [11]:
# img_metas (list[dict]): List of image info dict where each dict
#                 has: 'img_shape', 'scale_factor', 'flip', and may also contain
#                 'filename', 'ori_shape', 'pad_shape', and 'img_norm_cfg'.